# echuu v2 - AI VTuber 直播引擎（带完整剧本+记忆系统）

## 核心架构

```
标注数据 → Pattern Analyzer
               ↓
用户输入 (角色设定 + 话题)
               ↓
┌──────────────────────────────────────────────────────┐
│  Phase 1: 预生成完整剧本 (100s)                       │
│  • 10-15句完整台词，有起承转合                        │
│  • 每句标注 key_info（关键信息点）                   │
│  • 设置 interruption_cost                            │
└──────────────────────────────────────────────────────┘
               ↓
┌──────────────────────────────────────────────────────┐
│  Phase 2: 实时表演 + 记忆系统                         │
│                                                       │
│  🧠 记忆系统                                          │
│  ├─ 剧本进度（说到第几句）                            │
│  ├─ 已提到信息（避免重复）                            │
│  ├─ 弹幕记忆（已回应/待回答）                         │
│  └─ 承诺追踪（说了"等会告诉你"要兑现）                │
│                                                       │
│  ⚡ 统一弹幕处理                                      │
│  priority = base + relevance_bonus + sc_bonus        │
│  • 相关性是最重要因素                                 │
│  • 高相关问题 > 低相关SC                              │
│                                                       │
│  🔍 答案查找机制                                      │
│  在 key_info 中匹配问题 → 决定叙事策略                │
│  • 答案还没说：吊胃口 + 记录承诺                      │
│  • 答案在1-3句后：可跳跃（高优先级）                  │
│  • 答案已说过：提醒观众                               │
└──────────────────────────────────────────────────────┘
               ↓
     输出: 台词 + 🔊 语音 + 记忆状态可视化
```

## 核心创新

1. **完整剧本预生成** - 100s完整故事，避免车轱辘话
2. **key_info 标注** - 每句话的关键信息，用于匹配弹幕问题
3. **记忆系统** - 追踪已说内容、弹幕、承诺
4. **统一弹幕处理** - 相关性为王的优先级系统
5. **承诺追踪** - 说了"等会告诉你"必须兑现
6. **答案查找** - 在剧本中定位问题答案，决定叙事策略

## Part 1: 环境准备

In [ ]:
# 安装依赖
!pip install anthropic dashscope python-dotenv -q

In [ ]:
import json
import os
import sys
import re
import random
from pathlib import Path
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple, Callable
from enum import Enum
from collections import defaultdict, Counter
from datetime import datetime
import io

from IPython.display import display, HTML, Markdown, Audio

# 加载环境变量
from dotenv import load_dotenv

# 自动查找项目根目录
PROJECT_ROOT = Path.cwd()
while PROJECT_ROOT.name != 'echuu-agent' and PROJECT_ROOT.parent != PROJECT_ROOT:
    PROJECT_ROOT = PROJECT_ROOT.parent

load_dotenv(PROJECT_ROOT / '.env')

print(f"📁 项目根目录: {PROJECT_ROOT}")
print(f"🔑 ANTHROPIC_API_KEY: {'✅ 已配置' if os.getenv('ANTHROPIC_API_KEY') else '❌ 未配置'}")
print(f"🔑 DASHSCOPE_API_KEY: {'✅ 已配置' if os.getenv('DASHSCOPE_API_KEY') else '❌ 未配置'}")
print(f"\n✅ 导入完成（已添加 re, random 等V2所需模块）")

## Part 2: 数据结构定义

In [ ]:
# ==================== 核心数据结构 ====================

@dataclass
class ScriptLine:
    """剧本台词（带key_info标注）"""
    id: str                          # 唯一ID
    text: str                        # 台词内容
    stage: str                       # Hook/Build-up/Climax/Resolution
    interruption_cost: float         # 0.0-1.0
    key_info: List[str] = field(default_factory=list)  # 🔑 关键信息点（用于匹配弹幕）
    
    def __repr__(self):
        info_str = ", ".join(self.key_info[:2])
        return f"[{self.stage}] {self.text[:30]}... (keys: {info_str})"


@dataclass
class Danmaku:
    """弹幕（带优先级计算）"""
    text: str
    user: str = "观众"
    is_sc: bool = False
    amount: int = 0              # SC金额
    
    # 计算得出的属性
    relevance: float = 0.0       # 与当前话题的相关性 0.0-1.0
    priority: float = 0.0        # 最终优先级
    
    @classmethod
    def from_text(cls, text: str) -> "Danmaku":
        """解析弹幕"""
        is_sc = False
        amount = 0
        
        # 检测SC
        if "SC" in text or "¥" in text or "$" in text:
            is_sc = True
            # 简单提取金额
            import re
            match = re.search(r'[¥$]?\s*(\d+)', text)
            if match:
                amount = int(match.group(1))
        
        return cls(text=text, is_sc=is_sc, amount=amount)
    
    def is_question(self) -> bool:
        """判断是否是问题"""
        return "?" in self.text or "？" in self.text


@dataclass
class PerformerMemory:
    """
    记忆系统 - 可视化展示AI记住了什么
    """
    
    # 📖 剧本进度
    script_progress: Dict = field(default_factory=lambda: {
        "current_line": 0,
        "total_lines": 0,
        "completed_stages": [],
        "current_stage": "Hook",
    })
    
    # 💬 弹幕记忆
    danmaku_memory: Dict = field(default_factory=lambda: {
        "received": [],          # 收到的所有弹幕
        "responded": [],         # 已回应的
        "ignored": [],           # 被忽略的
        "pending_questions": [], # 待回答的问题
    })
    
    # 🤝 承诺追踪
    promises: List[Dict] = field(default_factory=list)
    # 例如: {"content": "室友生不生气", "made_at_step": 3, "fulfilled": False, "answer_at_line": 9}
    
    # 🎭 故事要点
    story_points: Dict = field(default_factory=lambda: {
        "mentioned": [],         # 已经提到的信息
        "upcoming": [],          # 即将要说的关键点
        "revealed": [],          # 已揭示的"答案"
    })
    
    # 😊 情绪轨迹
    emotion_track: List[Dict] = field(default_factory=list)
    
    def to_display(self) -> str:
        """生成用户可见的记忆状态"""
        lines = []
        lines.append("┌─────────────────────────────────────────────┐")
        lines.append("│ 🧠 AI记忆状态                               │")
        lines.append("├─────────────────────────────────────────────┤")
        
        # 剧本进度
        prog = self.script_progress
        current = prog.get("current_line", 0)
        total = prog.get("total_lines", 0)
        if total > 0:
            percent = int(current / total * 10)
            bar = "█" * percent + "░" * (10 - percent)
            lines.append(f"│ 📖 剧本: [{bar}] {current}/{total} ({prog.get('current_stage', '?')})  │")
        
        # 弹幕记忆
        dm = self.danmaku_memory
        responded = len(dm.get("responded", []))
        pending = len(dm.get("pending_questions", []))
        lines.append(f"│ 💬 弹幕: 已回应{responded}条, 待回答{pending}个问题         │")
        
        # 承诺
        unfulfilled = [p for p in self.promises if not p.get("fulfilled", False)]
        if unfulfilled:
            lines.append("│ 🤝 待兑现承诺:                              │")
            for p in unfulfilled[:2]:
                content = p.get("content", "")[:20]
                lines.append(f"│    • {content}...                         │")
        
        # 已提到要点
        mentioned = self.story_points.get("mentioned", [])
        if mentioned:
            lines.append("│ 🎭 已提到:                                  │")
            for point in mentioned[-3:]:
                lines.append(f"│    ✓ {point[:20]}...                      │")
        
        lines.append("└─────────────────────────────────────────────┘")
        return "\n".join(lines)
    
    def to_context(self) -> str:
        """生成给LLM的上下文摘要"""
        parts = []
        
        # 剧本进度
        prog = self.script_progress
        parts.append(f"剧本进度: {prog.get('current_line')}/{prog.get('total_lines')} ({prog.get('current_stage')})")
        
        # 已提到的信息
        mentioned = self.story_points.get("mentioned", [])
        if mentioned:
            parts.append(f"已提到: {', '.join(mentioned[-5:])}")
        
        # 待回答问题
        pending = self.danmaku_memory.get("pending_questions", [])
        if pending:
            parts.append(f"待回答: {', '.join(q[:20] for q in pending[:3])}")
        
        # 未兑现承诺
        unfulfilled = [p for p in self.promises if not p.get("fulfilled", False)]
        if unfulfilled:
            parts.append(f"待兑现承诺: {', '.join(p['content'][:15] for p in unfulfilled[:2])}")
        
        return " | ".join(parts)


@dataclass
class PerformanceState:
    """表演状态"""
    name: str
    persona: str
    background: str
    topic: str
    
    # 完整剧本（预生成）
    script_lines: List[ScriptLine] = field(default_factory=list)
    current_line_idx: int = 0
    current_step: int = 0
    
    # 记忆系统
    memory: PerformerMemory = field(default_factory=PerformerMemory)
    
    # 弹幕队列
    danmaku_queue: List[Danmaku] = field(default_factory=list)
    
    # 口癖
    catchphrases: List[str] = field(default_factory=list)

## Part 2.5: 弹幕处理核心（统一优先级系统）


In [ ]:
# ==================== 工具函数 ====================

def extract_keywords(text: str) -> List[str]:
    """提取关键词（简单版）"""
    import re
    # 移除标点
    text = re.sub(r'[，。！？、：；""''（）【】《》…~]', ' ', text)
    # 分词（简单按空格）
    words = text.split()
    # 过滤停用词
    stopwords = {'的', '了', '在', '是', '我', '你', '他', '她', '它', '们', 
                 '这', '那', '有', '和', '就', '不', '也', '都', '说', '很',
                 '吗', '吧', '呢', '啊', '哦', '嗯', '哈', '呀'}
    keywords = [w for w in words if len(w) >= 2 and w not in stopwords]
    return keywords[:5]  # 最多5个


# ==================== 弹幕处理器 ====================

class DanmakuEvaluator:
    """
    弹幕评估器 - 计算优先级
    
    优先级公式:
    priority = base_score + relevance_bonus + sc_bonus
    """
    
    def evaluate(self, danmaku: Danmaku, state: PerformanceState) -> Danmaku:
        """评估单条弹幕"""
        
        # 1. 基础分
        if danmaku.is_question():
            base = 0.3  # 问题类
        elif any(kw in danmaku.text for kw in ["哈哈", "笑死", "真的假的", "！", "牛"]):
            base = 0.15  # 情绪反馈类
        else:
            base = 0.1  # 普通评论
        
        # 2. 相关性评估
        relevance = self._calc_relevance(danmaku.text, state)
        danmaku.relevance = relevance
        
        if relevance > 0.7:
            relevance_bonus = 0.4  # 高相关
        elif relevance > 0.4:
            relevance_bonus = 0.2  # 中相关
        else:
            relevance_bonus = 0.0  # 低相关
        
        # 3. SC加成
        if danmaku.is_sc:
            if danmaku.amount >= 200:
                sc_bonus = 0.7
            elif danmaku.amount >= 100:
                sc_bonus = 0.5
            elif danmaku.amount >= 50:
                sc_bonus = 0.3
            else:
                sc_bonus = 0.2
        else:
            sc_bonus = 0.0
        
        danmaku.priority = base + relevance_bonus + sc_bonus
        
        return danmaku
    
    def _calc_relevance(self, text: str, state: PerformanceState) -> float:
        """计算弹幕与当前上下文的相关性"""
        # 提取当前故事的关键词
        story_keywords = set()
        for info in state.memory.story_points.get("mentioned", []):
            story_keywords.update(extract_keywords(info))
        for info in state.memory.story_points.get("upcoming", []):
            story_keywords.update(extract_keywords(info))
        
        # 如果没有故事关键词，检查话题
        if not story_keywords:
            story_keywords.update(extract_keywords(state.topic))
        
        # 提取弹幕关键词
        danmaku_keywords = set(extract_keywords(text))
        
        # 计算重叠度
        if not story_keywords:
            return 0.0
        overlap = len(danmaku_keywords & story_keywords)
        return min(overlap / 2, 1.0)  # 2个关键词重叠就算高相关


class DanmakuHandler:
    """
    统一弹幕处理器
    
    所有弹幕走同一流程，根据优先级决定：
    1. 是否打断
    2. 是否复读
    3. 如何回应
    4. 是否改变叙事
    """
    
    def __init__(self, evaluator: DanmakuEvaluator):
        self.evaluator = evaluator
    
    def handle(self, danmaku: Danmaku, state: PerformanceState) -> Dict:
        """
        处理弹幕
        
        Returns:
            {
                "should_interrupt": bool,
                "echo": str,
                "response": str,
                "action": str,  # continue/jump/tease/improvise
            }
        """
        
        # 评估优先级
        danmaku = self.evaluator.evaluate(danmaku, state)
        
        # 获取当前cost
        if state.current_line_idx >= len(state.script_lines):
            current_cost = 0.2  # 结尾阶段
        else:
            current_line = state.script_lines[state.current_line_idx]
            current_cost = current_line.interruption_cost
        
        # 决策：是否打断
        should_interrupt = danmaku.priority > current_cost
        
        if not should_interrupt:
            return {
                "should_interrupt": False,
                "action": "ignore",
                "priority": danmaku.priority,
                "cost": current_cost
            }
        
        # 决定是否复读
        echo = self._maybe_echo(danmaku)
        
        # 查找答案位置
        answer_loc = self._find_answer(danmaku.text, state)
        
        # 生成回应策略
        action = self._decide_action(danmaku, answer_loc, state)
        
        return {
            "should_interrupt": True,
            "echo": echo,
            "action": action,
            "answer_loc": answer_loc,
            "priority": danmaku.priority,
            "cost": current_cost,
            "relevance": danmaku.relevance
        }
    
    def _maybe_echo(self, danmaku: Danmaku) -> str:
        """决定是否复读弹幕"""
        if danmaku.is_sc:
            return f"诶有SC！有人说：{danmaku.text}"
        elif danmaku.priority > 0.5:
            return f"有人说{danmaku.text}，"
        elif danmaku.is_question():
            return f"有人问{danmaku.text}，"
        else:
            return ""  # 不复读
    
    def _find_answer(self, question: str, state: PerformanceState) -> Dict:
        """在剧本中查找问题的答案位置"""
        keywords = extract_keywords(question)
        current_idx = state.current_line_idx
        
        for i, line in enumerate(state.script_lines):
            if i < current_idx:
                continue  # 跳过已经说过的
            
            for info in line.key_info:
                if any(kw in info for kw in keywords):
                    return {
                        "found": True,
                        "line_idx": i,
                        "distance": i - current_idx,
                        "answer_hint": info,
                    }
        
        return {"found": False}
    
    def _decide_action(self, danmaku: Danmaku, answer_loc: Dict, 
                       state: PerformanceState) -> str:
        """
        决定叙事动作
        
        - continue: 回应后继续当前剧本
        - jump: 跳跃到答案位置
        - reveal: 直接揭晓答案
        - tease: 吊胃口
        - improvise: 即兴回答
        """
        
        if not answer_loc.get("found", False):
            return "improvise"
        
        distance = answer_loc.get("distance", 0)
        
        # 根据优先级和距离决定策略
        if danmaku.priority > 0.8:
            # 高优先级（大额SC或高相关问题）
            if distance <= 3:
                return "jump"
            else:
                return "tease"
        elif danmaku.priority > 0.5:
            # 中优先级
            if distance <= 2:
                return "continue"
            else:
                return "tease"
        else:
            # 低优先级
            return "continue"


# 初始化
evaluator = DanmakuEvaluator()
danmaku_handler = DanmakuHandler(evaluator)

print("✅ 弹幕处理器已初始化")
print("  • 优先级公式: priority = base + relevance_bonus + sc_bonus")
print("  • 相关性是最重要因素")


## Part 2.6: 剧本生成器（预生成完整台词）


In [ ]:
class ScriptGeneratorV2:
    """
    剧本生成器 V2 - 预生成完整台词（带 key_info 标注）
    """
    
    SYSTEM_PROMPT = """你是VTuber剧本编剧。生成**场景化台词**，组成完整故事。

⚠️ **数量要求：必须生成15-18句台词**（参考clip_005标准：403秒/15-18个叙事单元）

## 🔥 关键：必须是场景化叙事，不要总结式！

### ❌ 错误示范（总结式，太干瘪）
```
"那时候我刚到日本，穷得叮当响"
"室友是富二代，有很多零食"
"我偷吃了室友的腰果，很紧张"
"后来坦白了，室友很宽容"
```
↑ 这种像在做摘要，缺乏细节和内心戏，**绝对不要这样写！**

### ✅ 正确示范（场景化，有质感）
```
"我那个时候一个月给自己订的，一个月喝三瓶可乐，因为我特别爱喝可乐，但是当时没钱"
  → 具体数字、生活细节、口语化

"我看着那个冰箱里的，他的那个海苔炒腰果，我好馋呀。我想着我吃一个腰果，他回来他又不数，他应该不知道吧"
  → 内心独白、犹豫过程、场景描写

"第二天又想着，要不我再偷吃一个吧。结果我吃着吃着，他还没回来，我已经把里面的腰果吃完了"
  → 时间推进、行动升级、自然推进

"把我紧张的都睡不着觉了，每天晚上我都没睡着，因为当我意识到，我在里头一翻腾，发现腰果没了"
  → 情绪描写、具体动作、心理负担

"他说'你想吃就吃呀随便吃呀'。我说真的吗？我一想哇，这么慷慨我真的可以随便吃吗？他说'对呀，你想吃我再给你拿一盒来啊'"
  → 完整对话、内心反应、情绪释放
```

## 写作技巧清单（每句必备）

✅ **必须包含的元素**：
1. **具体数字/细节** - "一个月三瓶可乐" 不要 "很少"
2. **内心独白** - "我心想"、"我想着"、"哎呀"、"我一想"
3. **情绪描写** - "紧张的睡不着"、"心理负担可重了"、"我好馋呀"
4. **生活质感** - 酱油拌饭、冰箱、汇率、房间、东西的名字
5. **对话场景** - 写完整对话，不要转述"他说了..."
6. **时间推进** - "第二天"、"结果"、"后来"、"那个时候"
7. **心路历程** - 犹豫→行动→后悔→释然，展示变化过程

✅ **语言风格**：
- 口语化、有停顿、自我修正（"三瓶还是四瓶"、"对吧"、"就是说"）
- 允许重复强调（"真的吗？真的吗？"、"天哪"）
- 用"然后"、"结果"、"后来"连接，像在聊天
- 展示思考过程，不要直接给结论

✅ **叙事节奏**：
- 不要跳过过程！要展示心路历程
- 铺垫要充分（先说穷→可乐限量→室友富→看到腰果→起贪念）
- 高潮要有张力（第一次偷吃→第二天又吃→发现吃完→失眠→坦白→释然）
- 用内心戏拉长时间感

## 叙事结构
- Hook (2-3句): 悬念/问题开场，0.2-0.4 cost
- Build-up (4-5句): 铺垫背景，积累情绪，0.5-0.7 cost
- Climax (3-4句): 高潮转折，揭晓答案，0.8-0.9 cost
- Resolution (2-3句): 总结升华，0.3-0.5 cost

## key_info 标注（用于匹配弹幕问题）

"我那个时候一个月给自己订的，一个月喝三瓶可乐"
key_info: ["留学穷困", "可乐限量", "节约生活"]

"我想我吃一个腰果，他回来他又不数，他应该不知道吧"
key_info: ["第一次偷吃", "侥幸心理", "内心挣扎"]

"他说你想吃就吃呀随便吃呀"
key_info: ["室友反应", "室友宽容", "没生气", "危机解除"]  ← 这是"室友生气吗"的答案！"""
    
    def __init__(self, llm: LLMClient, analyzer: PatternAnalyzer = None):
        self.llm = llm
        self.analyzer = analyzer
    
    def generate(self, name: str, persona: str, background: str, 
                 topic: str, language: str = "zh") -> List[ScriptLine]:
        """生成完整剧本"""
        
        # 获取参考数据
        catchphrases = ""
        example_hooks = ""
        
        if self.analyzer:
            cps = self.analyzer.extract_catchphrases(language)[:5]
            catchphrases = ", ".join(f'"{c}"' for c, _ in cps)
            
            hooks = self.analyzer.extract_hooks(language)[:2]
            example_hooks = "\n".join(f'- "{h[:50]}..."' for h in hooks)
        
        prompt = f"""为 {name} 生成关于"{topic}"的场景化故事剧本。

## 角色信息
- 名字: {name}
- 人设: {persona}
- 背景: {background}
- 话题: {topic}
- 口癖: {catchphrases or "自然即可"}

## 开场参考
{example_hooks or "（无）"}

## ⚠️ 数量要求（严格遵守！）

**必须生成 15-18 句台词**（参考 clip_005 标准）

结构分配：
- **Hook（开场）**: 3句 - 悬念/问题开场，吸引注意
- **Build-up（铺垫）**: 6-7句 - 详细铺垫背景，大量生活细节和内心戏
- **Climax（高潮）**: 4-5句 - 转折、情绪爆发、揭晓答案
- **Resolution（收尾）**: 2-3句 - 情绪释放、总结升华

⚠️ **少于15句会导致叙事不够丰富、细节不足！请严格生成15-18句！**

## 写作要求（每句必备）

1. **具体数字/细节** - "一个月三瓶可乐" 不要 "很少"
2. **内心独白** - "我心想"、"我想着"、"哎呀"
3. **情绪描写** - "紧张的睡不着"、"心理负担可重了"
4. **生活质感** - 酱油拌饭、冰箱、汇率、物品名
5. **完整对话** - 写出来，不要转述
6. **时间推进** - "第二天"、"结果"、"后来"

### 不要写总结式（❌）
❌ "那时候我很穷" 
✅ "我那个时候一个月给自己订的，一个月喝三瓶可乐，三瓶还是四瓶，因为我特别爱喝，但是当时没钱"

## 输出格式

JSON数组：
[
  {{"text": "台词（要长！有细节！有内心戏！）", "stage": "Hook", "cost": 0.3, "key_info": ["信息1", "信息2"]}},
  ...至少12个元素...
]

要求：
- **严格生成15-18句（参考clip_005密度）**
- 每句推进新信息，不重复
- key_info准确（用于匹配弹幕）
- 符合{name}人设和口癖
- 台词要长，有场景感，有细节

⚠️ 如果生成少于15句，说明细节不够丰富，请重新生成！

只输出JSON数组。"""
        
        response = self.llm.call(prompt, system=self.SYSTEM_PROMPT, max_tokens=4500)
        
        try:
            # 解析JSON
            if "```" in response:
                json_str = response.split("```")[1]
                if json_str.startswith("json"):
                    json_str = json_str[4:]
            else:
                json_str = response
            
            lines_data = json.loads(json_str.strip())
            
            # 转换为 ScriptLine
            script_lines = []
            for i, line_data in enumerate(lines_data):
                script_lines.append(ScriptLine(
                    id=f"line_{i}",
                    text=line_data.get("text", ""),
                    stage=line_data.get("stage", "Unknown"),
                    interruption_cost=line_data.get("cost", 0.5),
                    key_info=line_data.get("key_info", [])
                ))
            
            print(f"✅ 生成了 {len(script_lines)} 句台词")
            return script_lines
            
        except Exception as e:
            print(f"[Script] 解析失败: {e}")
            print(f"响应内容: {response[:200]}...")
            return self._fallback_script(name, topic)
    
    def _fallback_script(self, name: str, topic: str) -> List[ScriptLine]:
        """后备剧本（15句标准）"""
        return [
            # Hook (3句)
            ScriptLine("line_0", f"大家好，今天我想和你们聊聊{topic}", "Hook", 0.3, [topic]),
            ScriptLine("line_1", "这是一个很有意思的故事，你们肯定想不到会发生什么", "Hook", 0.3, ["开场悬念"]),
            ScriptLine("line_2", "让我慢慢讲给你们听，先从背景说起", "Hook", 0.4, ["引入"]),
            # Build-up (6句)
            ScriptLine("line_3", "故事是这样的，那个时候我还...", "Build-up", 0.5, ["背景"]),
            ScriptLine("line_4", "我记得特别清楚，当时的情况是这样的", "Build-up", 0.6, ["铺垫"]),
            ScriptLine("line_5", "有一个很重要的细节，我得告诉你们", "Build-up", 0.6, ["细节"]),
            ScriptLine("line_6", "然后呢，事情开始变得有点不一样了", "Build-up", 0.7, ["转折前兆"]),
            ScriptLine("line_7", "我当时心里想，这事儿可能要不太妙", "Build-up", 0.7, ["内心戏"]),
            ScriptLine("line_8", "结果果然，接下来发生了一件事", "Build-up", 0.7, ["即将高潮"]),
            # Climax (4句)
            ScriptLine("line_9", "当时我整个人都傻了，你们猜发生了什么？", "Climax", 0.8, ["高潮开始"]),
            ScriptLine("line_10", "就在那一瞬间，关键的转折来了", "Climax", 0.9, ["转折"]),
            ScriptLine("line_11", "我告诉你们，结果完全出乎意料", "Climax", 0.8, ["揭晓"]),
            ScriptLine("line_12", "没想到最后竟然是这样，哈哈哈", "Climax", 0.7, ["反应"]),
            # Resolution (3句)
            ScriptLine("line_13", "现在想起来还是很感慨，那次经历真的很特别", "Resolution", 0.4, ["感悟"]),
            ScriptLine("line_14", "后来我就明白了一个道理", "Resolution", 0.3, ["升华"]),
            ScriptLine("line_15", "你们有类似经历吗？可以在弹幕告诉我~", "Resolution", 0.3, ["互动"]),
        ]


print("✅ 剧本生成器 V2 已定义（预生成完整台词模式）")


## Part 2.7: 表演引擎 V2（带记忆系统）


In [ ]:
class PerformerV2:
    """
    表演引擎 V2 - 带记忆系统和统一弹幕处理
    """
    
    def __init__(self, llm: LLMClient, tts: TTSClient, danmaku_handler: DanmakuHandler):
        self.llm = llm
        self.tts = tts
        self.danmaku_handler = danmaku_handler
    
    def step(self, state: PerformanceState, new_danmaku: List[Danmaku] = None) -> Dict:
        """
        执行一步表演
        
        流程：
        1. 评估弹幕优先级
        2. 决定是否打断当前台词
        3. 如果打断：生成回应 + 承接回剧本
        4. 如果不打断：说当前台词
        5. 更新记忆
        6. 检查承诺是否该兑现
        """
        
        if new_danmaku:
            state.danmaku_queue.extend(new_danmaku)
            # 记录到记忆
            for d in new_danmaku:
                state.memory.danmaku_memory["received"].append(d.text)
        
        # 检查结束
        if state.current_line_idx >= len(state.script_lines):
            return self._generate_ending(state)
        
        current_line = state.script_lines[state.current_line_idx]
        
        # 处理弹幕
        best_danmaku = None
        handle_result = None
        
        if state.danmaku_queue:
            # 找到最高优先级的弹幕
            for danmaku in state.danmaku_queue:
                result = self.danmaku_handler.handle(danmaku, state)
                if result.get("should_interrupt"):
                    if best_danmaku is None or result.get("priority", 0) > handle_result.get("priority", 0):
                        best_danmaku = danmaku
                        handle_result = result
        
        # 决策
        if best_danmaku and handle_result:
            # 打断！回应弹幕
            output = self._handle_danmaku_response(best_danmaku, handle_result, current_line, state)
            
            # 从队列移除
            state.danmaku_queue = [d for d in state.danmaku_queue if d != best_danmaku]
            
            # 记录到记忆
            state.memory.danmaku_memory["responded"].append(best_danmaku.text)
            if best_danmaku.is_question():
                # 如果是问题，检查是否记录承诺
                if handle_result.get("action") == "tease":
                    answer_loc = handle_result.get("answer_loc", {})
                    if answer_loc.get("found"):
                        state.memory.promises.append({
                            "content": best_danmaku.text,
                            "made_at_step": state.current_step,
                            "fulfilled": False,
                            "answer_at_line": answer_loc.get("line_idx")
                        })
        else:
            # 继续当前台词
            output = {
                "speech": current_line.text,
                "action": "continue",
                "priority": 0.0,
                "cost": current_line.interruption_cost
            }
        
        # 更新剧本进度
        state.current_line_idx += 1
        state.current_step += 1
        
        # 更新记忆：记录已提到的信息
        for info in current_line.key_info:
            if info not in state.memory.story_points["mentioned"]:
                state.memory.story_points["mentioned"].append(info)
        
        # 更新剧本进度记忆
        state.memory.script_progress["current_line"] = state.current_line_idx
        state.memory.script_progress["total_lines"] = len(state.script_lines)
        state.memory.script_progress["current_stage"] = current_line.stage
        
        # 检查承诺是否兑现
        self._check_promises(current_line, state)
        
        # 生成语音
        speech = output.get("speech", "")
        audio = None
        if speech and self.tts.enabled:
            audio = self.tts.synthesize(speech)
        
        output["audio"] = audio
        output["line_idx"] = state.current_line_idx - 1
        output["stage"] = current_line.stage
        output["step"] = state.current_step
        output["memory_display"] = state.memory.to_display()
        
        return output
    
    def _handle_danmaku_response(self, danmaku: Danmaku, handle_result: Dict,
                                  current_line: ScriptLine, state: PerformanceState) -> Dict:
        """处理弹幕回应"""
        
        echo = handle_result.get("echo", "")
        action = handle_result.get("action", "continue")
        answer_loc = handle_result.get("answer_loc", {})
        
        # 生成回应内容
        response_part = ""
        
        if action == "improvise":
            # 即兴回答（剧本里没有）
            response_part = "哈哈这个问题问得好！虽然不在今天的话题里，但让我想想..."
        elif action == "jump":
            # 跳跃到答案（高优先级）
            response_part = f"既然你问了，我直接告诉你！"
            # TODO: 可以考虑跳跃到答案行
        elif action == "tease":
            # 吊胃口
            distance = answer_loc.get("distance", 0)
            if distance <= 2:
                response_part = "诶你别急，马上就要说到了~"
            else:
                response_part = "这个问题太好了！先卖个关子，你们继续听就知道了，绝对出乎意料~"
        else:  # continue
            response_part = "说得对！"
        
        # 承接回剧本
        transition = self._generate_transition(current_line)
        
        speech = f"{echo}{response_part} {transition}{current_line.text}"
        
        return {
            "speech": speech,
            "action": action,
            "danmaku": danmaku.text,
            "priority": handle_result.get("priority", 0),
            "cost": handle_result.get("cost", 0),
            "relevance": handle_result.get("relevance", 0)
        }
    
    def _generate_transition(self, next_line: ScriptLine) -> str:
        """生成承接语"""
        transitions = [
            "好，那刚才说到，",
            "回到我们的故事，",
            "继续说，",
            "对了，",
            ""  # 有时候不需要过渡
        ]
        import random
        return random.choice(transitions)
    
    def _check_promises(self, current_line: ScriptLine, state: PerformanceState):
        """检查当前台词是否兑现了承诺"""
        for promise in state.memory.promises:
            if promise.get("fulfilled", False):
                continue
            
            # 检查当前行是不是答案行
            if promise.get("answer_at_line") == state.current_line_idx - 1:
                promise["fulfilled"] = True
                # 可以在台词中加入提醒："诶对了，刚才有人问XXX，答案来了！"
    
    def _generate_ending(self, state: PerformanceState) -> Dict:
        """生成结尾"""
        speech = f"好啦，今天关于{state.topic}就聊到这里，谢谢大家！"
        audio = self.tts.synthesize(speech) if self.tts.enabled else None
        
        return {
            "speech": speech,
            "action": "end",
            "step": state.current_step,
            "audio": audio,
            "memory_display": state.memory.to_display()
        }


print("✅ 表演引擎 V2 已定义（带记忆系统和统一弹幕处理）")


## Part 2.8: 完整引擎 V2


In [ ]:
class EchuuEngineV2:
    """
    echuu 引擎 V2 - 完整流程
    
    Phase 1: 预生成完整剧本
    Phase 2: 实时表演 + 记忆系统
    """
    
    def __init__(self, llm: LLMClient, tts: TTSClient, analyzer: PatternAnalyzer):
        self.llm = llm
        self.tts = tts
        self.analyzer = analyzer
        self.script_gen = ScriptGeneratorV2(llm, analyzer)
        self.performer = PerformerV2(llm, tts, danmaku_handler)
    
    def create_performance(self, name: str, persona: str, background: str,
                          topic: str, language: str = "zh") -> PerformanceState:
        """
        Phase 1: 创建表演（预生成完整剧本）
        """
        print(f"\n{'='*60}")
        print(f"🎬 echuu v2 - 预生成完整剧本")
        print(f"{'='*60}\n")
        
        print(f"角色: {name}")
        print(f"话题: {topic}\n")
        
        print("⏳ 正在生成100秒完整剧本...")
        script_lines = self.script_gen.generate(name, persona, background, topic, language)
        
        # 显示剧本
        print(f"\n📖 生成的剧本：")
        print(f"{'='*60}")
        for i, line in enumerate(script_lines):
            cost_bar = "█" * int(line.interruption_cost * 5) + "░" * (5 - int(line.interruption_cost * 5))
            print(f"\n[{i}] {line.stage} {cost_bar} cost={line.interruption_cost:.1f}")
            print(f"    {line.text[:80]}{'...' if len(line.text) > 80 else ''}")
            print(f"    🔑 key_info: {', '.join(line.key_info)}")
        print(f"\n{'='*60}\n")
        
        # 提取口癖
        catchphrases = [cp for cp, _ in self.analyzer.extract_catchphrases(language)[:5]]
        
        # 初始化记忆
        memory = PerformerMemory()
        memory.script_progress["total_lines"] = len(script_lines)
        memory.script_progress["current_stage"] = script_lines[0].stage if script_lines else "Unknown"
        
        # 提取即将要说的关键点
        for line in script_lines:
            memory.story_points["upcoming"].extend(line.key_info)
        
        return PerformanceState(
            name=name,
            persona=persona,
            background=background,
            topic=topic,
            script_lines=script_lines,
            memory=memory,
            catchphrases=catchphrases
        )
    
    def run(self, state: PerformanceState, danmaku_sim: List[Dict] = None,
            play_audio: bool = True) -> List[Dict]:
        """
        Phase 2: 运行表演（实时模式）
        """
        
        results = []
        danmaku_by_step = defaultdict(list)
        
        if danmaku_sim:
            for d in danmaku_sim:
                step = d.get("step", 0)
                danmaku_by_step[step].append(Danmaku.from_text(d.get("text", "")))
        
        print(f"\n{'='*60}")
        print(f"🎭 开始实时表演")
        print(f"{'='*60}\n")
        
        for step in range(len(state.script_lines)):
            # 获取新弹幕
            new_danmaku = danmaku_by_step.get(step, [])
            
            # 执行一步
            result = self.performer.step(state, new_danmaku)
            results.append(result)
            
            # 显示
            step_num = result.get("step", 0)
            stage = result.get("stage", "?")
            action = result.get("action", "continue")
            speech = result.get("speech", "")
            
            action_icons = {
                "continue": "📖",
                "tease": "🎣",
                "jump": "⚡",
                "improvise": "🎲",
                "end": "🎭"
            }
            icon = action_icons.get(action, "📖")
            
            print(f"[Step {step_num}] {stage} {icon} {action.upper()}")
            print(f"  📢 {speech[:100]}{'...' if len(speech) > 100 else ''}")
            
            if result.get("danmaku"):
                print(f"  💬 回应弹幕: {result['danmaku']}")
                print(f"  📊 priority={result.get('priority', 0):.2f}, cost={result.get('cost', 0):.2f}, relevance={result.get('relevance', 0):.2f}")
            
            # 显示记忆（每3步显示一次）
            if step_num % 3 == 0:
                print(f"\n{result.get('memory_display', '')}")
            
            # 播放音频
            if play_audio and result.get('audio'):
                print(f"  🔊 播放语音...")
                display(Audio(result['audio'], autoplay=True))
            
            print()
            
            if action == "end":
                break
        
        print(f"\n{'='*60}")
        print(f"✅ 表演结束！")
        print(f"{'='*60}\n")
        
        # 最终记忆状态
        print("🧠 最终记忆状态：")
        print(state.memory.to_display())
        
        return results


# 初始化引擎
engine_v2 = EchuuEngineV2(llm, tts, analyzer)
print("\n✅ echuu 引擎 V2 初始化完成！")


## Part 9: 🚀 测试 echuu V2（完整剧本+记忆系统）


In [ ]:
# 测试用例：经典的室友腰果故事

test_case_v2 = {
    "name": "六螺",
    "persona": "25岁主播，活泼自嘲，喜欢分享生活经历，口癖：我觉得、对吧、就是说",
    "background": "做过很多工作，现在是全职主播，留学日本多年",
    "topic": "留学时偷吃室友腰果的故事",
    "danmaku": [
        {"step": 1, "text": "哈哈哈"},
        {"step": 2, "text": "我也有类似经历"},
        {"step": 4, "text": "室友知道吗？"},  # 高相关问题
        {"step": 6, "text": "[SC ¥50] 后来室友生气了吗"},  # SC + 高相关
        {"step": 8, "text": "笑死"},
    ]
}

# Phase 1: 预生成完整剧本
state_v2 = engine_v2.create_performance(
    name=test_case_v2["name"],
    persona=test_case_v2["persona"],
    background=test_case_v2["background"],
    topic=test_case_v2["topic"],
    language="zh"
)


In [ ]:
# Phase 2: 运行实时表演（带弹幕互动）

results_v2 = engine_v2.run(
    state_v2, 
    danmaku_sim=test_case_v2["danmaku"],
    play_audio=True  # 设为 False 可以禁用自动播放
)


## 📊 V2 版本核心改进

### 问题解决对比

| 问题 | V1（旧版） | V2（新版） |
|------|-----------|-----------|
| **车轱辘话** | 实时生成，容易重复 | ✅ 预生成完整剧本，每句推进新信息 |
| **叙事延展性** | 节点骨架，内容模糊 | ✅ 10-15句完整台词，有起承转合 |
| **弹幕承接** | 简单 urgency 判断 | ✅ 优先级公式（相关性为王） |
| **问题匹配** | 无法定位答案 | ✅ key_info 匹配答案位置 |
| **承诺追踪** | 说了"等会"就忘了 | ✅ 记忆系统追踪并兑现承诺 |
| **重复回答** | 不知道说过什么 | ✅ 记忆记录已提到信息 |

### 核心创新

1. **完整剧本预生成**
   - 100秒完整故事，避免车轱辘话
   - 每句带 `key_info` 标注关键信息

2. **统一弹幕处理**
   ```
   priority = base + relevance_bonus + sc_bonus
   ```
   - 高相关问题 (0.7) > 低相关SC¥50 (0.4)
   - 相关性是最重要因素

3. **记忆系统**
   - 📖 剧本进度
   - 💬 弹幕记忆（已回应/待回答）
   - 🤝 承诺追踪
   - 🎭 故事要点（已提到/即将揭晓）

4. **答案查找机制**
   - 在 `key_info` 中匹配问题关键词
   - 根据距离决定策略：
     - 1-3句后 → 跳跃（高优先级）
     - 4+句后 → 吊胃口 + 承诺
     - 已说过 → 提醒观众

### 优先级矩阵示例

**场景**: 正在讲 "室友是富二代，零食很多"（Build-up, cost=0.5）

| 弹幕 | base | relevance | SC | priority | 结果 |
|------|------|-----------|----|---------| -----|
| "哈哈哈" | 0.15 | 0.0 | 0.0 | 0.15 | ❌ 不打断(0.15 < 0.5) |
| "室友知道吗" | 0.3 | 0.4 (高相关) | 0.0 | **0.7** | ✅ 打断！查找答案 |
| "[SC¥50] 室友生气吗" | 0.3 | 0.4 | 0.3 | **0.9** | ✅ 打断！吊胃口+承诺 |
| "主播多大" | 0.3 | 0.0 | 0.0 | 0.3 | ❌ 不打断(低相关) |


## 🎨 自定义测试


In [ ]:
# 🎨 自定义你的角色和话题！

my_test = {
    "name": "小梅",  # 👈 修改名字
    "persona": "温柔的猫娘主播，说话带喵~，喜欢分享温暖的故事",  # 👈 修改人设
    "background": "大学毕业，现在是全职主播",  # 👈 修改背景
    "topic": "第一次养猫的经历",  # 👈 修改话题
    "danmaku": [
        {"step": 1, "text": "好可爱~"},
        {"step": 3, "text": "猫咪叫什么名字？"},  # 高相关问题
        {"step": 5, "text": "[SC ¥100] 猫咪现在多大了"},  # 大额SC
        {"step": 7, "text": "我也想养猫"},
    ]
}

# Phase 1: 生成剧本
my_state = engine_v2.create_performance(
    name=my_test["name"],
    persona=my_test["persona"],
    background=my_test["background"],
    topic=my_test["topic"],
    language="zh"
)


In [ ]:
# Phase 2: 运行表演
my_results = engine_v2.run(
    my_state,
    danmaku_sim=my_test["danmaku"],
    play_audio=True
)


---

## 📚 使用指南

### 快速开始

```python
# 1. 创建表演（预生成剧本）
state = engine_v2.create_performance(
    name="角色名",
    persona="人设描述",
    background="背景",
    topic="话题",
    language="zh"
)

# 2. 运行表演（带弹幕）
results = engine_v2.run(
    state,
    danmaku_sim=[
        {"step": 1, "text": "弹幕内容"},
        {"step": 3, "text": "[SC ¥50] 问题"},
    ],
    play_audio=True
)
```

### 核心机制

#### 1. 优先级公式
```python
priority = base_score + relevance_bonus + sc_bonus

# base_score
问题类: 0.3
情绪反馈: 0.15
普通评论: 0.1

# relevance_bonus（最重要！）
高相关 (0.7+): +0.4
中相关 (0.4-0.7): +0.2
低相关 (0.0-0.4): +0.0

# sc_bonus
¥200+: +0.7
¥100+: +0.5
¥50+: +0.3
其他: +0.2
```

#### 2. 决策规则
```python
if priority > current_line.interruption_cost:
    # 打断！回应弹幕
    if 问题答案在剧本中:
        if 距离 <= 3句:
            action = "jump"  # 跳跃（高优先级）
        else:
            action = "tease"  # 吊胃口 + 记录承诺
    else:
        action = "improvise"  # 即兴回答
else:
    # 继续当前台词
    action = "continue"
```

#### 3. 记忆更新时机
- ✅ 说完每句台词 → 记录 `key_info` 到 `story_points.mentioned`
- ✅ 回应弹幕 → 记录到 `danmaku_memory.responded`
- ✅ 吊胃口 → 记录承诺到 `promises`
- ✅ 到达答案行 → 标记承诺为 `fulfilled`

### 关键特性

| 特性 | 说明 | 解决的问题 |
|------|------|-----------|
| 🎬 **完整剧本预生成** | 10-15句完整台词 | 车轱辘话、叙事不延展 |
| 🔑 **key_info 标注** | 每句标注关键信息 | 无法匹配问题和答案 |
| 🧠 **记忆系统** | 追踪已说内容、弹幕、承诺 | 重复、遗忘承诺 |
| ⚡ **相关性为王** | 高相关 > 低相关SC | 弹幕不能引导话题 |
| 🎣 **吊胃口+承诺** | 答案还没说就承诺稍后揭晓 | 打断叙事节奏 |
| 🔊 **实时语音** | CosyVoice TTS | 完整的直播体验 |

### 调试技巧

```python
# 查看生成的剧本
for i, line in enumerate(state.script_lines):
    print(f"[{i}] {line.text}")
    print(f"    key_info: {line.key_info}")

# 查看记忆状态
print(state.memory.to_display())

# 查看弹幕优先级
danmaku = Danmaku.from_text("室友生气了吗？")
evaluated = evaluator.evaluate(danmaku, state)
print(f"priority={evaluated.priority}, relevance={evaluated.relevance}")
```

### 音色切换

```python
# 切换音色
tts.change_voice("longyingjing_v3")  # 温柔女声
tts.change_voice("longxiaochun_v2")  # 活泼女声
tts.change_voice("longanyang")       # 自然男声
```

---

## 🎯 下一步优化方向

1. **LLM 辅助回应生成**：目前回应是模板化的，可以让 LLM 根据人设生成更自然的回应
2. **跳跃实现**：高优先级弹幕可以真正跳跃到答案行
3. **承诺兑现提醒**：到达答案行时主动提醒"刚才有人问..."
4. **情绪追踪**：根据剧本内容和弹幕追踪情绪变化
5. **WebSocket 实时弹幕**：接入真实直播平台的弹幕流


---

## ⚡ 快速运行流程

### 第一次使用

1. **运行所有 cell（按顺序）**：
   - Part 1: 环境准备 ✓
   - Part 2: 数据结构定义 ✓
   - Part 2.5-2.8: 核心组件（弹幕处理、剧本生成、表演引擎）✓
   - Part 3-8: Pattern Analyzer、TTS、LLM 初始化 ✓
   - Part 9: 测试 V2 引擎 ← **从这里开始**

2. **测试预设案例**（Part 9）：
   ```python
   # 运行 cell 15: 生成剧本
   state_v2 = engine_v2.create_performance(...)
   
   # 运行 cell 16: 实时表演
   results_v2 = engine_v2.run(state_v2, ...)
   ```

3. **自定义测试**（Part 10）：
   ```python
   # 修改 my_test 字典中的内容
   # 运行 cell 19: 生成剧本
   # 运行 cell 20: 实时表演
   ```

### 调整参数

```python
# 禁用自动播放
results = engine_v2.run(state, danmaku_sim=..., play_audio=False)

# 修改音色
tts.change_voice("longyingjing_v3")

# 查看剧本详情
for i, line in enumerate(state.script_lines):
    print(f"[{i}] {line.stage}: {line.text}")
    print(f"    key_info: {line.key_info}\n")
```

### 核心改进验证

运行测试后，注意观察：

✅ **车轱辘话解决** → 每句台词推进新信息，有起承转合  
✅ **弹幕承接** → 高相关问题被优先回应  
✅ **记忆可视化** → 每3步显示记忆状态  
✅ **承诺追踪** → 说了"等会告诉你"会被记录  
✅ **答案查找** → 根据 key_info 匹配问题和答案位置


## Part 3: Pattern Analyzer - 从真实数据学习

In [ ]:
class PatternAnalyzer:
    """从标注数据中提取模式"""
    
    def __init__(self, annotated_clips: List[Dict]):
        self.clips = annotated_clips
        self.all_segments = []
        for clip in annotated_clips:
            self.all_segments.extend(clip.get("segments", []))
    
    def _normalize_field(self, value, default="self"):
        """处理字段值，如果是列表则取第一个元素"""
        if isinstance(value, list):
            return value[0] if value else default
        return value if value else default
    
    def compute_attention_transitions(self) -> Dict[str, Dict[str, float]]:
        """计算attention转移概率"""
        trans = defaultdict(lambda: defaultdict(int))
        for clip in self.clips:
            segs = clip.get("segments", [])
            for i in range(len(segs) - 1):
                f = self._normalize_field(segs[i].get("attention_focus"), "self")
                t = self._normalize_field(segs[i+1].get("attention_focus"), "self")
                trans[f][t] += 1
        
        prob = {}
        for f, tos in trans.items():
            total = sum(tos.values())
            prob[f] = {t: c/total for t, c in tos.items()}
        return prob
    
    def infer_baseline_costs(self) -> Dict[str, float]:
        """推断不同attention下的打断代价"""
        focus_stats = defaultdict(lambda: {"total": 0, "ignored": 0})
        
        for seg in self.all_segments:
            focus = self._normalize_field(seg.get("attention_focus"), "self")
            trigger = self._normalize_field(seg.get("trigger"), "self")
            act = self._normalize_field(seg.get("speech_act"), "narrate")
            
            if trigger == "danmaku":
                focus_stats[focus]["total"] += 1
                if act != "respond":
                    focus_stats[focus]["ignored"] += 1
        
        costs = {}
        for focus, stats in focus_stats.items():
            if stats["total"] > 0:
                costs[focus] = stats["ignored"] / stats["total"]
            else:
                costs[focus] = 0.5
        return costs
    
    def extract_skeletons(self) -> List[Tuple[str, int]]:
        """提取叙事骨架"""
        skeletons = [c.get("skeleton", "") for c in self.clips if c.get("skeleton")]
        return Counter(skeletons).most_common(10)
    
    def extract_catchphrases(self, language: str = None) -> List[Tuple[str, int]]:
        """提取口癖"""
        cps = []
        for c in self.clips:
            if language and c.get("language") != language:
                continue
            cps.extend(c.get("catchphrases", []))
        return Counter(cps).most_common(20)
    
    def extract_hooks(self, language: str = None) -> List[str]:
        """提取开场示例"""
        hooks = []
        for c in self.clips:
            if language and c.get("language") != language:
                continue
            segs = c.get("segments", [])
            if segs:
                hooks.append(segs[0].get("text", "")[:100])
        return hooks[:10]
    
    def get_report(self) -> str:
        """生成分析报告"""
        lines = [
            "="*50,
            "📊 Pattern Analysis Report",
            "="*50,
            f"Total clips: {len(self.clips)}, Total segments: {len(self.all_segments)}",
        ]
        
        lines.append("\n### Attention Transitions")
        for f, tos in self.compute_attention_transitions().items():
            top = sorted(tos.items(), key=lambda x: -x[1])[:3]
            lines.append(f"- `{f}` → " + ", ".join(f"`{t}`:{p:.0%}" for t, p in top))
        
        lines.append("\n### Inferred Interruption Costs")
        for focus, cost in self.infer_baseline_costs().items():
            bar = "█" * int(cost * 10) + "░" * (10 - int(cost * 10))
            lines.append(f"- `{focus}`: {bar} {cost:.2f}")
        
        lines.append("\n### Top Catchphrases")
        cps = self.extract_catchphrases()[:10]
        lines.append(", ".join(f'"{c}"({n})' for c, n in cps))
        
        return "\n".join(lines)

## Part 4: 加载标注数据

In [ ]:
# 加载标注数据
DATA_PATH = PROJECT_ROOT / "data" / "annotated_clips.json"

with open(DATA_PATH, "r", encoding="utf-8") as f:
    annotated_clips = json.load(f)

print(f"✅ 加载了 {len(annotated_clips)} 个clips")

# 创建分析器
analyzer = PatternAnalyzer(annotated_clips)

# 显示报告
display(Markdown(analyzer.get_report()))

In [ ]:
# 查看示例数据
print("\n📝 开场 Hook 示例 (中文):")
for i, hook in enumerate(analyzer.extract_hooks("zh")[:3], 1):
    print(f"  {i}. {hook[:60]}...")

print("\n📝 叙事骨架示例:")
for skel, count in analyzer.extract_skeletons()[:5]:
    print(f"  [{count}次] {skel[:50]}...")

## Part 5: TTS Client - 通义千问语音合成

In [ ]:
class TTSClient:
    """通义千问 CosyVoice TTS 客户端"""
    
    def __init__(self):
        self.enabled = False
        self.synthesizer_class = None
        self.audio_format = None
        
        api_key = os.getenv("DASHSCOPE_API_KEY")
        if api_key:
            try:
                import dashscope
                from dashscope.audio.tts_v2 import SpeechSynthesizer, AudioFormat
                
                dashscope.api_key = api_key
                self.synthesizer_class = SpeechSynthesizer
                self.audio_format = AudioFormat.MP3_22050HZ_MONO_256KBPS
                self.model = os.getenv("TTS_MODEL", "cosyvoice-v3-flash")
                self.voice = os.getenv("TTS_VOICE", "longanyang")
                self.enabled = True
                print(f"✅ TTS 已启用: model={self.model}, voice={self.voice}")
            except ImportError:
                print("⚠️ dashscope 未安装")
        else:
            print("⚠️ 未设置 DASHSCOPE_API_KEY，TTS 已禁用")
    
    def synthesize(self, text: str) -> Optional[bytes]:
        """合成语音，返回音频数据"""
        if not self.enabled:
            return None
        
        try:
            synthesizer = self.synthesizer_class(
                model=self.model,
                voice=self.voice,
                format=self.audio_format
            )
            return synthesizer.call(text)
        except Exception as e:
            print(f"[TTS] 错误: {e}")
            return None
    
    def change_voice(self, voice: str):
        """切换音色"""
        self.voice = voice
        print(f"🔊 音色已切换为: {voice}")


# 初始化 TTS
tts = TTSClient()

In [ ]:
# 测试 TTS
if tts.enabled:
    print("🎤 测试 TTS...")
    test_audio = tts.synthesize("你好，我是echuu AI主播，今天来给大家讲个有趣的故事！")
    if test_audio:
        print(f"✅ 生成音频: {len(test_audio)} bytes")
        display(Audio(test_audio, autoplay=False))
else:
    print("⚠️ TTS 未启用，跳过测试")

## Part 6: LLM Client - Claude

In [ ]:
class LLMClient:
    """Claude LLM 客户端"""
    
    def __init__(self):
        self.client = None
        self.model = os.getenv("DEFAULT_MODEL", "claude-3-haiku-20240307")
        
        api_key = os.getenv("ANTHROPIC_API_KEY")
        if api_key:
            try:
                import anthropic
                self.client = anthropic.Anthropic(api_key=api_key)
                print(f"✅ LLM 已初始化: {self.model}")
            except ImportError:
                print("⚠️ anthropic 未安装")
        else:
            print("⚠️ 未设置 ANTHROPIC_API_KEY，使用 Mock 模式")
    
    def call(self, prompt: str, system: str = None, max_tokens: int = 1000) -> str:
        """调用 LLM"""
        if self.client:
            try:
                kwargs = {
                    "model": self.model,
                    "max_tokens": max_tokens,
                    "messages": [{"role": "user", "content": prompt}]
                }
                if system:
                    kwargs["system"] = system
                
                response = self.client.messages.create(**kwargs)
                return response.content[0].text
            except Exception as e:
                print(f"[LLM] 错误: {e}")
        
        return self._mock_response(prompt)
    
    def _mock_response(self, prompt: str) -> str:
        """Mock 响应"""
        import random
        
        if "JSON数组" in prompt or "script" in prompt.lower():
            return '''[
{"stage": "Hook", "goal": "用悬念开场", "attention": "audience", "speech_act": "elicit", "duration": 20, "cost": 0.3, "hint": "提问开场"},
{"stage": "Build-up", "goal": "铺垫背景", "attention": "self", "speech_act": "narrate", "duration": 40, "cost": 0.6, "hint": "描述情况"},
{"stage": "Climax", "goal": "关键转折", "attention": "self", "speech_act": "narrate", "duration": 40, "cost": 0.9, "hint": "情感爆发"},
{"stage": "Resolution", "goal": "总结感悟", "attention": "audience", "speech_act": "opine", "duration": 20, "cost": 0.4, "hint": "分享心得"}
]'''
        else:
            return json.dumps({
                "inner_monologue": random.choice(["让我想想...", "有意思...", "继续...", "嗯..."]),
                "decision": "continue",
                "speech": random.choice(["说起这个事儿啊...", "你们知道吗...", "我跟你们讲..."]),
                "emotion": "neutral"
            }, ensure_ascii=False)


# 初始化 LLM
llm = LLMClient()

## Part 7: Script Generator - 剧本生成

In [ ]:
class ScriptGenerator:
    """剧本生成器"""
    
    SYSTEM_PROMPT = """你是一个专业的VTuber剧本编剧。根据角色设定和话题，设计2分钟的表演剧本骨架。

叙事结构：
- Hook: 用问题/悬念/共鸣点开场
- Build-up: 铺垫背景、积累情绪
- Climax: 情感爆发点、关键信息
- Resolution: 总结升华、和观众连接

interruption_cost（被弹幕打断的代价）：
- 0.0-0.3: 可以随时打断（闲聊、开场）
- 0.4-0.6: 最好不打断（铺垫阶段）
- 0.7-0.9: 尽量不打断（高潮阶段）
- 0.9+: 绝对不能打断（情感爆发）"""
    
    def __init__(self, llm: LLMClient, analyzer: PatternAnalyzer = None):
        self.llm = llm
        self.analyzer = analyzer
    
    def generate(self, name: str, persona: str, background: str, 
                 topic: str, language: str = "zh") -> List[NarrativeNode]:
        """生成剧本"""
        
        # 获取参考数据
        example_skeletons = ""
        catchphrases = ""
        
        if self.analyzer:
            skels = self.analyzer.extract_skeletons()[:3]
            example_skeletons = "\n".join(f"- {s}" for s, _ in skels)
            
            cps = self.analyzer.extract_catchphrases(language)[:5]
            catchphrases = ", ".join(f'"{c}"' for c, _ in cps)
        
        prompt = f"""请为以下VTuber设计一个2分钟的直播剧本骨架：

## 角色信息
- 名字: {name}
- 人设: {persona}
- 背景: {background}
- 今日话题: {topic}

## 参考叙事骨架
{example_skeletons or "（无）"}

## 可用口癖
{catchphrases or "（无）"}

## 输出要求
输出一个JSON数组，每个元素包含：
- stage: 阶段 (Hook/Build-up/Climax/Resolution)
- goal: 目标
- attention: 注意力 (self/audience/specific)
- speech_act: 言语行为 (narrate/opine/respond/elicit)
- duration: 时长（秒）
- cost: 打断代价 (0.0-1.0)
- hint: 内容提示

设计4-6个节点，总时长约120秒。只输出JSON。"""
        
        response = self.llm.call(prompt, system=self.SYSTEM_PROMPT)
        
        try:
            if "```" in response:
                json_str = response.split("```")[1]
                if json_str.startswith("json"):
                    json_str = json_str[4:]
            else:
                json_str = response
            
            nodes_data = json.loads(json_str.strip())
            
            return [
                NarrativeNode(
                    stage=n.get("stage", "Unknown"),
                    goal=n.get("goal", ""),
                    target_attention=n.get("attention", "self"),
                    target_speech_act=n.get("speech_act", "narrate"),
                    duration_sec=n.get("duration", 30),
                    interruption_cost=n.get("cost", 0.5),
                    content_hint=n.get("hint", "")
                )
                for n in nodes_data
            ]
        except Exception as e:
            print(f"[Script] 解析失败: {e}")
            return self._fallback_script(topic)
    
    def _fallback_script(self, topic: str) -> List[NarrativeNode]:
        """后备剧本"""
        return [
            NarrativeNode("Hook", f"引出{topic}", "audience", "elicit", 20, 0.3),
            NarrativeNode("Build-up", "铺垫背景", "self", "narrate", 40, 0.6),
            NarrativeNode("Climax", "核心内容", "self", "narrate", 40, 0.9),
            NarrativeNode("Resolution", "总结升华", "audience", "opine", 20, 0.4),
        ]


# 初始化
script_gen = ScriptGenerator(llm, analyzer)

## Part 8: Cognitive Performer - 认知表演者（核心）

In [ ]:
class CognitivePerformer:
    """认知表演者 - 实时决策和生成"""
    
    SYSTEM_PROMPT = """你是一个VTuber的"内心"。每个时刻需要：
1. 感知当前状态（剧本进度、弹幕、时间）
2. 做出决策（继续剧本/回应弹幕/即兴/沉默）
3. 输出内心独白（这会显示给观众看，是核心特色）
4. 生成台词

关键：内心独白要展示你的"思考过程"，让观众感受到AI的agency。
- 不是假装像人，而是让观众看到你在"思考"和"选择"
- "选择不回应"也是一种表达
- 内心独白要简短、真实、有personality"""
    
    def __init__(self, llm: LLMClient, tts: TTSClient, analyzer: PatternAnalyzer = None):
        self.llm = llm
        self.tts = tts
        self.analyzer = analyzer
    
    def step(self, state: PerformanceState, new_danmaku: List[Danmaku] = None) -> Dict:
        """
        执行一次认知循环
        
        核心公式: decision_value = urgency - cost
        - 正数 → 倾向回应弹幕
        - 负数 → 倾向继续剧本
        """
        
        if new_danmaku:
            state.danmaku_queue.extend(new_danmaku)
        
        # 检查结束
        if state.current_node_idx >= len(state.script):
            return self._generate_ending(state)
        
        current_node = state.script[state.current_node_idx]
        elapsed_ratio = state.node_elapsed_sec / current_node.duration_sec if current_node.duration_sec > 0 else 1.0
        current_cost = current_node.get_current_cost(elapsed_ratio)
        
        # 评估弹幕
        max_urgency = 0.0
        most_urgent = None
        for d in state.danmaku_queue:
            if d.urgency > max_urgency:
                max_urgency = d.urgency
                most_urgent = d
        
        # 构建 prompt
        recent_danmaku = [d.text for d in state.danmaku_queue[-5:]]
        catchphrases_hint = ", ".join(state.catchphrases[:3]) if state.catchphrases else "无"
        
        prompt = f"""## 当前状态

### 角色
名字: {state.name}
人设: {state.persona}
口癖参考: {catchphrases_hint}

### 剧本进度
当前阶段: {current_node.stage}
阶段目标: {current_node.goal}
阶段进度: {elapsed_ratio:.0%}
内容提示: {current_node.content_hint}
话题: {state.topic}

### 叙事代价
当前cost: {current_cost:.2f} (0=随意打断, 1=绝对不能打断)

### 弹幕情况
最近弹幕: {recent_danmaku or "（无）"}
最高紧急度: {max_urgency:.2f}
连续忽略数: {state.ignored_count}

### 决策参考
urgency - cost = {max_urgency - current_cost:.2f}
正数倾向回应，负数倾向继续
连续忽略3条以上应考虑补救

## 输出
输出JSON: {{"inner_monologue": "...", "decision": "continue/respond", "speech": "...", "emotion": "..."}}
- inner_monologue: 内心独白（会显示给观众）
- speech: 台词（会转成语音）
- emotion: 情绪 (neutral/excited/sad/focused/engaged)

只输出JSON。"""
        
        response = self.llm.call(prompt, system=self.SYSTEM_PROMPT, max_tokens=500)
        
        # 解析
        try:
            if "```" in response:
                json_str = response.split("```")[1]
                if json_str.startswith("json"):
                    json_str = json_str[4:]
            else:
                json_str = response
            result = json.loads(json_str.strip())
        except:
            result = {
                "inner_monologue": f"继续讲{current_node.stage}...",
                "decision": "continue",
                "speech": f"说到{state.topic}...",
                "emotion": "neutral"
            }
        
        # 更新状态
        decision = result.get("decision", "continue")
        step_duration = 10
        state.node_elapsed_sec += step_duration
        state.total_elapsed_sec += step_duration
        
        if state.node_elapsed_sec >= current_node.duration_sec:
            state.current_node_idx += 1
            state.node_elapsed_sec = 0
        
        if decision == "respond" and most_urgent:
            state.danmaku_queue = [d for d in state.danmaku_queue if d != most_urgent]
            state.ignored_count = 0
            result["target_danmaku"] = most_urgent.text
        elif decision == "continue" and state.danmaku_queue:
            state.ignored_count += 1
        
        # 生成语音
        speech = result.get("speech", "")
        audio = None
        if speech and self.tts.enabled:
            audio = self.tts.synthesize(speech)
        
        result["audio"] = audio
        result["node"] = current_node.stage
        result["time"] = f"{state.total_elapsed_sec:.0f}s"
        result["cost"] = current_cost
        result["urgency"] = max_urgency
        result["delta"] = max_urgency - current_cost
        
        return result
    
    def _generate_ending(self, state: PerformanceState) -> Dict:
        """生成结尾"""
        speech = f"好啦，今天关于{state.topic}就聊到这里，下次见！"
        audio = self.tts.synthesize(speech) if self.tts.enabled else None
        
        return {
            "inner_monologue": "该收尾了~",
            "decision": "end",
            "speech": speech,
            "emotion": "satisfied",
            "node": "END",
            "time": f"{state.total_elapsed_sec:.0f}s",
            "audio": audio
        }


# 初始化
performer = CognitivePerformer(llm, tts, analyzer)

## Part 9: echuu Engine - 完整引擎

In [ ]:
class EchuuEngine:
    """echuu 核心引擎"""
    
    def __init__(self, llm: LLMClient, tts: TTSClient, analyzer: PatternAnalyzer):
        self.llm = llm
        self.tts = tts
        self.analyzer = analyzer
        self.script_gen = ScriptGenerator(llm, analyzer)
        self.performer = CognitivePerformer(llm, tts, analyzer)
    
    def create_performance(self, name: str, persona: str, background: str,
                          topic: str, language: str = "zh") -> PerformanceState:
        """创建表演"""
        print(f"\n🎬 正在生成剧本...")
        script = self.script_gen.generate(name, persona, background, topic, language)
        
        # 提取参考数据
        catchphrases = [cp for cp, _ in self.analyzer.extract_catchphrases(language)[:5]]
        hooks = self.analyzer.extract_hooks(language)[:3]
        
        return PerformanceState(
            name=name,
            persona=persona,
            background=background,
            topic=topic,
            script=script,
            catchphrases=catchphrases,
            example_hooks=hooks
        )
    
    def run(self, state: PerformanceState, danmaku_sim: List[Dict] = None,
            max_steps: int = 12, play_audio: bool = True) -> List[Dict]:
        """运行表演"""
        
        results = []
        danmaku_by_step = defaultdict(list)
        if danmaku_sim:
            for d in danmaku_sim:
                step = d.get("step", 0)
                danmaku_by_step[step].append(Danmaku.from_text(d.get("text", "")))
        
        print(f"\n{'='*60}")
        print(f"🎭 {state.name} - {state.topic}")
        print(f"{'='*60}")
        
        print(f"\n📋 剧本骨架:")
        for i, node in enumerate(state.script):
            cost_bar = "█" * int(node.interruption_cost * 5) + "░" * (5 - int(node.interruption_cost * 5))
            print(f"  [{i+1}] {node.stage} {cost_bar} cost={node.interruption_cost:.1f}")
            print(f"      → {node.goal}")
        
        print(f"\n🎬 开始表演...\n")
        
        for step in range(max_steps):
            new_danmaku = danmaku_by_step.get(step, [])
            result = self.performer.step(state, new_danmaku)
            results.append(result)
            
            # 显示
            time_str = result.get('time', '?')
            node_str = result.get('node', '?')
            decision = result.get('decision', 'continue').upper()
            
            dec_icon = {"RESPOND": "💬", "CONTINUE": "📖", "END": "🎭"}.get(decision, "🎭")
            
            print(f"[{time_str}] {node_str} {dec_icon}")
            print(f"  💭 {result.get('inner_monologue', '')}")
            print(f"  📢 {result.get('speech', '(沉默)')}")
            
            if 'delta' in result:
                delta = result['delta']
                delta_str = f"↑{delta:.2f}" if delta > 0 else f"↓{abs(delta):.2f}"
                print(f"  📊 urgency={result.get('urgency', 0):.2f} - cost={result.get('cost', 0):.2f} = {delta_str}")
            
            if result.get('target_danmaku'):
                print(f"  💬 回应: {result['target_danmaku']}")
            
            # 播放音频
            if play_audio and result.get('audio'):
                print(f"  🔊 播放语音...")
                display(Audio(result['audio'], autoplay=True))
            
            print()
            
            if result.get('decision') == "end":
                break
        
        print(f"{'='*60}")
        print(f"✅ 表演结束！总时长: {state.total_elapsed_sec:.0f}秒")
        print(f"{'='*60}")
        
        return results


# 初始化引擎
engine = EchuuEngine(llm, tts, analyzer)
print("\n✅ echuu Engine 初始化完成！")

## Part 10: 运行完整表演！

In [ ]:
# 测试用例: 基于真实切片的话题
test_case = {
    "name": "六螺",
    "persona": "25岁主播，活泼自嘲，喜欢分享生活经历，口癖：我觉得、对吧、就是说",
    "background": "做过很多工作，现在是全职主播，留学日本多年",
    "topic": "留学时偷吃室友腰果的故事",
    "danmaku": [
        {"step": 1, "text": "哈哈哈"},
        {"step": 2, "text": "我也有类似经历"},
        {"step": 4, "text": "[SC ¥50] 后来室友生气了吗"},
        {"step": 6, "text": "笑死"},
        {"step": 8, "text": "太真实了"},
    ]
}

# 创建表演
state = engine.create_performance(
    name=test_case["name"],
    persona=test_case["persona"],
    background=test_case["background"],
    topic=test_case["topic"],
    language="zh"
)

In [ ]:
# 运行表演（带模拟弹幕和语音）
# 设置 play_audio=True 自动播放语音，False 只生成不播放
results = engine.run(
    state, 
    danmaku_sim=test_case["danmaku"], 
    max_steps=10,
    play_audio=True  # 设为 False 可以禁用自动播放
)

## Part 11: 决策过程可视化

In [ ]:
# 可视化决策过程
print("\n📊 决策过程可视化:")
print("\n时间 | 阶段 | urgency | cost | Δ | 决策")
print("-" * 65)

for r in results:
    if r.get('decision') == 'end':
        continue
    
    time = r.get('time', '?')
    node = r.get('node', '?')[:10].ljust(10)
    urgency = r.get('urgency', 0)
    cost = r.get('cost', 0)
    delta = r.get('delta', urgency - cost)
    decision = r.get('decision', 'continue')
    
    urg_bar = "█" * int(urgency * 5) + "░" * (5 - int(urgency * 5))
    cost_bar = "█" * int(cost * 5) + "░" * (5 - int(cost * 5))
    
    dec_icon = "💬" if decision == "respond" else "📖"
    
    print(f"{time:>5} | {node} | {urg_bar} {urgency:.1f} | {cost_bar} {cost:.1f} | {delta:+.2f} | {dec_icon} {decision}")

## Part 12: 自定义测试

In [ ]:
# 🎨 自定义你的角色和话题！

my_character = {
    "name": "小猫咪",  # 👈 修改名字
    "persona": "软萌可爱的猫娘主播，说话带喵~，喜欢撒娇",  # 👈 修改人设
    "background": "从猫咪星球来的留学生",  # 👈 修改背景
    "topic": "第一次吃人类的猫粮（薯片）的体验",  # 👈 修改话题
}

my_danmaku = [
    {"step": 1, "text": "好可爱喵~"},
    {"step": 3, "text": "薯片好吃吗"},
    {"step": 5, "text": "[SC ¥100] 给你买更多零食！"},
]

# 运行
my_state = engine.create_performance(**my_character, language="zh")
my_results = engine.run(my_state, danmaku_sim=my_danmaku, max_steps=8, play_audio=True)

## Part 13: 切换音色

In [ ]:
# 可用音色列表
print("🔊 可用音色:")
print("  - longanyang    (龙小安，男声，自然)")
print("  - longyingjing_v3 (龙盈静，女声，温柔)")
print("  - longxiaochun_v2 (龙晓春，女声，活泼)")
print("  - longlaotie_v2   (龙老铁，男声，沉稳)")
print("\n更多音色: https://help.aliyun.com/zh/model-studio/cosyvoice-voice-list")

# 切换音色示例
# tts.change_voice("longyingjing_v3")

In [ ]:
# 测试不同音色
if tts.enabled:
    test_text = "大家好，我是你们的AI主播，今天来给大家讲个有趣的故事！"
    
    for voice in ["longanyang", "longyingjing_v3"]:
        tts.change_voice(voice)
        audio = tts.synthesize(test_text)
        if audio:
            print(f"\n🎤 {voice}:")
            display(Audio(audio, autoplay=False))

---

## 总结

### 核心机制

1. **Interruption Cost** - 从真实数据学习"被打断代价"
   - `self` 注意力时 cost 较高（不想被打断讲自己的事）
   - `audience` 注意力时 cost 较低（本来就在互动）
   - cost 随节点进度衰减（快讲完时可以转场）

2. **决策公式**
   ```
   decision_value = urgency - cost
   正数 → 回应弹幕
   负数 → 继续剧本
   ```

3. **Inner Monologue** - Killer Feature
   - 让观众看到AI的"思考过程"
   - "选择不回应"也是一种表达

4. **实时语音** - CosyVoice TTS
   - 低延迟流式合成
   - 多种音色可选